In [2]:
from Loader import fileloader as fl
from Loader import datahandledatabase as dhdb
import pandas as pd
import numpy as np
import time
from datetime import datetime
import openpyxl as xlsx

In [6]:
periodo = 201809
start_date_pre = '2018-07-16' # Only DDNN1
start_date = '2018-08-16' # Captura teams
end_date = '2018-09-16' # End of Captura teams

TESTPATH = 'D:/Datos de Usuario/cleon/Documents/Mercado Empresas/Data Fuente Comisiones/test/'

inidb = fl.ReadIniFile('C:/Anaconda3/MeScripts/Comisiones4/Config/mydbconfig.ini')
dbparser = inidb.readFile()

dbmanager = dhdb.DbDataProcess(str(periodo))
dbmanager.setParser(dbparser)

start_date_pre_dt = datetime.strptime(start_date_pre, '%Y-%m-%d')
start_date_dt = datetime.strptime(start_date, '%Y-%m-%d')
end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')

In [7]:
def textToDateConverter(df, date_cols):
# Text cols to dates converter    
    for datevalue in date_cols:
        df[datevalue] = pd.to_datetime(df[datevalue],'%Y-%m-%d', dayfirst = True)
    
    return df

In [8]:
# Data Wrangling Ventas y Deacs Voz
# ********************************************************
# Equipos quincenales : actualizar
start_time = time.time()

# PROCESO DE REPORTERIA SOBRE DESACTIVACIONES VOZ
#************************************************

COLS_DEAC = ['PERIODO_DESACTIVACION','GERENCIA1','GERENCIA2','ZONAVENTA','DEPARTAMENTO',
              'SUPERVISORKAM','VENDEDOR_ACTIVACION','VENDEDOR_OTRO','POSICION_EMPL',
              'FECHA_PROCESO','CANALDEVENTA','CODIGO','RAZON_SOCIAL','CONTRATO','FEC_ACTIV','FEC_DESACTIVA', 'TELEFONO',
              'ESTADO','PLAN_ACTIVACION','FECHA_ACTUALIZACION','PACK_CHIP','MOTIVO_DEAC','TIPODOC','DOCUMENTO',
              'CEDENTE_ACTIVACION','PORTABILIDAD',
              'TECNOLOGIAEQUIPO','ACCESS', 'ACCESSBOLSA','ACCESSPAQUETE','ACCESSLICENCIA','FECHA_CESE_EMPLEADO',
              'COMISION_UNITARIA','DIAS_DESACTIVADOS','RANGO_DESACTIVACION','CATEGORIA_MOTIVO_DEAC',
              'CATEGORIA_TECNOLOGIAEQUIPO','ACTIVIDAD', 'PENALIDAD','REVERSION','TIPO_REVERSION']

DATE_COLUMNS = ['FECHA_PROCESO','FEC_ACTIV','FEC_DESACTIVA']
DROP_COLUMNS = ['VENDEDOR_OTRO']
RENAME_COLUMNS ={'COMISION_UNITARIA':'COMISION_UNITARIA_DB'}
HALF_MONTHTLY_TEAMS = ['FALCO','MICHA','KRATOS','KAIZEN','DNEGOCIOS_EMPRESAS1']
HALF_MONTHTLY_TEAMS_PRE = ['DNEGOCIOS_EMPRESAS1']

CHANNEL_FILTER1 = ['PEQUEÑAS Y MEDIANAS EMPRESAS']
CHANNEL_FILTER2 = ['CORPORACIONES','GRANDES CLIENTES','VENTA REGIONAL EMPRESA']

ORDER_COLUMN = ['PERIODO_DESACTIVACION','GERENCIA1','GERENCIA2','DEPARTAMENTO','ZONAVENTA','SUPERVISORKAM',
                'VENDEDOR_ACTIVACION','FECHA_PROCESO','ESTADO']

deacs_ = dbmanager.loadData('View_Deacs')

deacs_ = textToDateConverter(deacs_, DATE_COLUMNS)
deacs_ = deacs_[COLS_DEAC]

deacs_.to_csv(TESTPATH + str(periodo) + '_deacs_.csv', index = False) # Punto de control 1

filter1 = deacs_['PERIODO_DESACTIVACION'] == start_date_dt.year * 100 + start_date_dt.month
filter2 = deacs_['PERIODO_DESACTIVACION'] == end_date_dt.year * 100 + end_date_dt.month
filter3 = deacs_['ZONAVENTA'].isin(HALF_MONTHTLY_TEAMS)
filter4 = deacs_['FECHA_PROCESO'] <= end_date_dt
filter5 = deacs_['FECHA_PROCESO'] >= start_date_dt
filter6 = deacs_['FECHA_PROCESO'] < start_date_dt # DDNN_1
filter7 = deacs_['ZONAVENTA'].isin(HALF_MONTHTLY_TEAMS_PRE)

deacs_t_0 = deacs_[filter6 & filter7]
deacs_t_1_1 = deacs_[filter1 & ~filter3]
deacs_t_1_2 = deacs_[filter1 & filter5 & filter3 & ~filter7]
deacs_t = deacs_[filter2 & filter4 & filter3 & ~filter7]
deacs = deacs_t_0.append(deacs_t_1_1, ignore_index = True)
deacs = deacs.append(deacs_t_1_2, ignore_index = True)
deacs = deacs.append(deacs_t, ignore_index = True) 

deacs.to_csv(TESTPATH + str(periodo) + '_deacs_1.csv', index = False)# Punto de control 2

stage1 = deacs['GERENCIA2'].isin(CHANNEL_FILTER2)
stage2 = deacs['GERENCIA1'].isin(CHANNEL_FILTER1)

# Filtrando las deacs_gce
#************************
#"""
deacs_gce = deacs[stage1]
# Corrigiendo los códigos de regiones múltiples
filter_reg = deacs_gce['VENDEDOR_OTRO'] != ''
#multiple_seller = deacs_gce[filter_reg]['VENDEDOR_OTRO'].values
#deacs_gce.loc[filter_reg,'VENDEDOR'] = multiple_seller # verificar si va

deacs_gce.drop(DROP_COLUMNS, axis =1, inplace = True)
deacs_gce = deacs_gce.rename(columns = RENAME_COLUMNS)

deacs_gce.sort(columns = ORDER_COLUMN,ascending = [1,1,1,1,1,1,1,1,1], inplace = True )
deacs_gce.to_csv(TESTPATH + str(periodo) + '_deacs_gce.csv', index = False)

#"""

# Filtrando las deacs_gcp
#************************

#"""
deacs_gcp = deacs[stage2]
deacs_gcp.drop(DROP_COLUMNS, axis =1, inplace = True)
deacs_gcp.sort(columns = ORDER_COLUMN,ascending = [1,1,1,1,1,1,1,1,1], inplace = True )
deacs_gcp.to_csv(TESTPATH + str(periodo) + '_deacs_gcp.csv', index = False)

#"""

time_consuming = (time.time() - start_time) # In seconds

print("--- %s seconds ---" % time_consuming)

C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\lib\site-packages\pandas\core\frame.py:2915: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  inplace=inplace, kind=kind, na_position=na_position)


Los registros de la tabla View_Deacs es 602 registros  
--- 8.62323522567749 seconds ---


In [5]:
# Data Wrangling Deacs Data
# ********************************************************
# Equipos quincenales : actualizar
start_time = time.time()

half_monthly_teams = ['FALCO','MICHA','KRATOS','KAIZEN']
end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')

DATE_COLUMNS_DATA = ['ACTION_DATE']

ORDER_COLUMN_DATA = ['PERIODO_ACTIVACION','GERENCIA1','GERENCIA2','DEPARTAMENTO','ZONAVENTA','SUPERVISORKAM',
                     'VENDEDOR_CROSS_SELLING','ACTION_DATE','ESTADO']

CHANNEL_FILTER1 = ['PEQUEÑAS Y MEDIANAS EMPRESAS','PLATAFORMAS COMERCIALES REGIONES']
CHANNEL_FILTER2 = ['CORPORACIONES','GRANDES CLIENTES','VENTA REGIONAL EMPRESA']
CHANNEL_FILTER3 = ['HOGAR']

DROP_COLUMNS = ['GERENCIA2','ZONAVENTA','VENDEDOR_OTRO']
DROP_COLUMNS2 = ['NEW_GERENCIA2','NEW_ZONAVENTA','VENDEDOR_OTRO']
RENAME_COLUMNS_DATA ={'NEW_GERENCIA2':'GERENCIA2','NEW_ZONAVENTA':'ZONAVENTA'}

COLS_DATA = ['PERIODO_ACTIVACION','GERENCIA1','GERENCIA2','NEW_GERENCIA2','ZONAVENTA','NEW_ZONAVENTA','DEPARTAMENTO',
             'SUPERVISORKAM','VENDEDOR_CROSS_SELLING','VENDEDOR_OTRO','POSICION_EMPL','ACTION_DATE','CANALDEVENTA',
             'CODIGO','RAZON_SOCIAL','CONTRATO','SERVICE','PHONENUMBER','ESTADO','FECHA_ACTUALIZACION','RUC','ACCESS_REAL',
             'TIPO2','GANADO_POR','EQUIPO','CUENTA','OBSERVACIONES']

# PROCESO DE REPORTERIA SOBRE DESACTIVACIONES DATA
#*************************************************

#deacs_ssaa = dbmanager.loadData('View_Deacs_SSAA')
#deacs_procesados.to_csv(testpath + month + '_deacs_procesados.csv', index = False)
#deacs_ssaa.to_csv(testpath + month + '_deacs_ssaa_procesados.csv', index = False)

## Revisar los contratos nulos en BD ##

time_consuming = (time.time() - start_time) # In seconds

print("--- %s seconds ---" % time_consuming)

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:249: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-vie

Los registros de la tabla View_Ventas_SSAA es 3606 registros  
--- 4.857300043106079 seconds ---


C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\lib\site-packages\pandas\core\frame.py:2915: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  inplace=inplace, kind=kind, na_position=na_position)


In [8]:
wb = xlsx.load_workbook(PATH + PERIODO + FILE)
type(wb)

openpyxl.workbook.Workbook

In [7]:
start_date_dt.year * 100 + start_date_dt.month

201807

In [6]:
start_date_dt.month

7